# Linear Frequency-Dependent PTO 

In [1]:
import os
import logging

import matplotlib.pyplot as plt
import capytaine as cpy
from autograd.builtins import isinstance, tuple, list, dict
import autograd.numpy as np

import wecopttool as wot


# logging 
logging.basicConfig(level=logging.INFO)

# create save directory
results_dir = 'results_linear_freqdep_pto'
if not os.path.exists(results_dir):
  os.makedirs(results_dir)

# frequencies
f0 = 0.05
nfreq = 50

### WEC

In [2]:
## WEC object
# mesh
wb = wot.geom.WaveBot() 
mesh_size_factor = 0.5 
mesh = wb.mesh(mesh_size_factor)

# floating body
fb = cpy.FloatingBody.from_meshio(mesh, name="WaveBot")
fb.add_translation_dof(name="HEAVE")

# hydrostatics
hs_data = wot.hydrostatics.hydrostatics(fb)
mass_33 = wot.hydrostatics.mass_matrix_constant_density(hs_data)[2, 2]
mass = np.atleast_2d(mass_33)
stiffness_33 = wot.hydrostatics.stiffness_matrix(hs_data)[2, 2]
stiffness = np.atleast_2d(stiffness_33)

# WEC object
wec = wot.WEC(fb, mass, stiffness, f0, nfreq)

# BEM
fname = os.path.join(results_dir, 'bem.nc')
if os.path.exists(fname):
    wec.read_bem(fname)
else:
    wec.run_bem()
    wec.write_bem(fname)


INFO:capytaine.bodies.bodies:Stored 1054 triangle faces as quadrilaterals
INFO:capytaine.bodies.bodies:New floating body: WaveBot.
INFO:wecopttool.core:Changing value of 'f_add'. This might cause some attributes to be reset.
INFO:wecopttool.core:New WEC: WaveBot with 1 DOF.
INFO:wecopttool.core:Reading BEM data from results_linear_freqdep_pto/bem.nc.
INFO:wecopttool.core:Impedance matrix deleted. To calculate impedance call 'self.bem_calc_impedance()'
INFO:wecopttool.core:Impedance matrix deleted. To calculate impedance call 'self.bem_calc_impedance()'
INFO:wecopttool.core:Calculating impedance matrix.
INFO:capytaine.post_pro.impedance:Compute impedance.


### PTO

In [45]:
# PTO kinematics
kinematics = np.eye(fb.nb_dofs)

# # PTO impedance
gear_ratio = 12.0
torque_constant = 6.7
winding_resistance = 0.5
winding_inductance = 0.0
drivetrain_inertia = 2.0 
drivetrain_friction = 1.0
drivetrain_stiffness = 0.0

drivetrain_impedance = (1j*wec.omega*drivetrain_inertia + 
                        drivetrain_friction + 
                        # -1/(wec.omega)*drivetrain_stiffness)  # Matlab code
                        1/(1j*wec.omega)*drivetrain_stiffness)  # Paper

winding_impedance = winding_resistance + 1j*wec.omega*winding_inductance

pto_impedance_11 = gear_ratio**2 * drivetrain_impedance
off_diag = np.sqrt(3.0/2.0) * torque_constant * gear_ratio
pto_impedance_12 = (-off_diag+0j) * np.ones(wec.omega.shape) # * -1 # TODO
pto_impedance_21 = (off_diag+0j) * np.ones(wec.omega.shape)
pto_impedance_22 = winding_impedance
pto_impedance = np.array([[pto_impedance_11, pto_impedance_12],
                          [pto_impedance_21, pto_impedance_22]])


# gear_ratio = 12.4666
# torque_constant = 6.1745
# electrical_constant = 4.116
# winding_resistance = 0.5

# pto_impedance_11 = (0.0+0j)*np.ones(wec.omega.shape)
# pto_impedance_12 = (torque_constant*gear_ratio+0j)*np.ones(wec.omega.shape)
# pto_impedance_21 = (electrical_constant*gear_ratio+0j)*np.ones(wec.omega.shape)
# pto_impedance_22 = (winding_resistance+0j)*np.ones(wec.omega.shape)
# pto_impedance_2 = np.array([[pto_impedance_11, pto_impedance_12],
#                           [pto_impedance_21, pto_impedance_22]])

# create PTO
pto = wot.pto.PseudoSpectralLinearPTO(nfreq, kinematics, pto_impedance)

# add PTO force to WEC
wec.f_add = pto.force_on_wec


INFO:wecopttool.core:Changing value of 'f_add'. This might cause some attributes to be reset.


### Waves

In [32]:
# waves
wfreq = 0.3
amplitude = 0.0625
phase = -40
waves = wot.waves.regular_wave(f0, nfreq, wfreq, amplitude, phase)

### Objective Function

In [33]:
# objective function
obj_fun = pto.electric_average_power
nstate_opt = pto.nstate

### Solve

In [30]:
# solve
scale_x_wec = 1.0
scale_x_opt = 1.0
scale_obj = 1.0

options = {'maxiter': 100, 'ftol': 1e-8}

wec_tdom, wec_fdom, x_wec, x_opt, obj, res = wec.solve(
    waves, obj_fun, nstate_opt, optim_options=options,
    scale_x_wec=scale_x_wec, scale_x_opt=scale_x_opt, scale_obj=scale_obj)

# post-process
pto_tdom, pto_fdom = pto.post_process(wec, x_wec, x_opt)

INFO:wecopttool.core:Solving pseudo-spectral control problem.
INFO:wecopttool.core:[mean(x_wec), mean(x_opt), obj_fun(x)]: [1.03e-01, 3.49e+01, 2.41e+06]
INFO:wecopttool.core:[mean(x_wec), mean(x_opt), obj_fun(x)]: [5.54e-01, 3.96e+01, 2.65e+07]
INFO:wecopttool.core:[mean(x_wec), mean(x_opt), obj_fun(x)]: [1.16e+02, 7.87e+03, 1.09e+12]
INFO:wecopttool.core:[mean(x_wec), mean(x_opt), obj_fun(x)]: [1.17e+05, 7.98e+06, 1.12e+18]
INFO:wecopttool.core:[mean(x_wec), mean(x_opt), obj_fun(x)]: [5.92e+08, 4.03e+10, 2.85e+25]
INFO:wecopttool.core:[mean(x_wec), mean(x_opt), obj_fun(x)]: [1.49e+13, 1.02e+15, 1.82e+34]
INFO:wecopttool.core:[mean(x_wec), mean(x_opt), obj_fun(x)]: [1.89e+18, 1.29e+20, 2.90e+44]
INFO:wecopttool.core:[mean(x_wec), mean(x_opt), obj_fun(x)]: [1.19e+24, 8.12e+25, 1.16e+56]
INFO:wecopttool.core:[mean(x_wec), mean(x_opt), obj_fun(x)]: [3.76e+30, 2.56e+32, 1.15e+69]
INFO:wecopttool.core:[mean(x_wec), mean(x_opt), obj_fun(x)]: [5.94e+37, 4.05e+39, 2.88e+83]
INFO:wecopttool.co

Iteration limit reached    (Exit mode 9)
            Current function value: nan
            Iterations: 100
            Function evaluations: 891
            Gradient evaluations: 100


### CC (Electrical) Solution


In [ ]:
Z_11 = pto_impedance[0, 0, :]
Z_12 = pto_impedance[0, 1, :]
Z_21 = pto_impedance[1, 0, :]
Z_22 = pto_impedance[1, 1, :]
# V_th = Z_21 / (Z_11 + Zi) * Fe
# Z_th = Z_22 - (Z_12*Z_21) / (Z_11 + Zi)
V_th = Z_21 / (Z_11 - Zi) * Fe
Z_th = Z_22 - (Z_12*Z_21) / (Z_11 - Zi)

cc_current_fd = V_th / (2*Z_th.real)
cc_voltage_fd = -1.0 * Z_th.conj() * cc_current_fd

cc_current_td = wot.post_process_continuous_time(cc_current_fd)
cc_voltage_td = wot.post_process_continuous_time(cc_voltage_fd)
